<a href="https://colab.research.google.com/github/Mrigakshi24-ux/Training-Sheet/blob/main/BoW%2C_Tf_IDF%2C_Word2Vec_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
doc1 = 'Game of Thrones is an amazing tv series!'
doc2 = 'Game of Thrones is the best tv series!'
doc3 = 'Game of Thrones is so great.'
doc4 = 'You guys should watch game of thrones, worth watching'

In [ ]:
# preprocessing
def preprocess(doc):
  # remove punctuation, convert to lower, return tokenized versionbn
  return re.sub(r"[^a-zA-Z0-9]", " ",doc.lower()).split()

In [ ]:
pre_doc1 = preprocess(doc1)
pre_doc2 = preprocess(doc2)
pre_doc3 = preprocess(doc3)
pre_doc4 = preprocess(doc4)

In [ ]:
# create vocab, taking union of all the four docs
vocab = np.union1d(pre_doc4, np.union1d(pre_doc3, np.union1d(pre_doc1, pre_doc2)))
vocab

array(['amazing', 'an', 'best', 'game', 'great', 'guys', 'is', 'of',
       'series', 'should', 'so', 'the', 'thrones', 'tv', 'watch',
       'watching', 'worth', 'you'], dtype='<U8')

In [ ]:
def returnVector(doc, vocab = vocab):
  l = [0]*len(vocab)
  for i,j in enumerate(vocab):
    if j in doc:
      l[i] = doc.count(j)
  return l

In [ ]:
vect1 = returnVector(pre_doc1)
vect2 = returnVector(pre_doc2)
vect3 = returnVector(pre_doc3)
vect4 = returnVector(pre_doc4)

In [ ]:
df = pd.DataFrame([vect1, vect2, vect3, vect4], columns= vocab )
df

,amazing,an,best,game,great,guys,is,of,series,should,so,the,thrones,tv,watch,watching,worth,you
0,1,1,0,1,0,0,1,1,1,0,0,0,1,1,0,0,0,0
1,0,0,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0
2,0,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0
3,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,1,1


In [ ]:
# using sklearn
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform([doc1, doc2, doc3, doc4])

print(vectorizer.get_feature_names_out())
print(X.toarray())

['amazing' 'an' 'best' 'game' 'great' 'guys' 'is' 'of' 'series' 'should'
 'so' 'the' 'thrones' 'tv' 'watch' 'watching' 'worth' 'you']
[[1 1 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0]
 [0 0 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0]
 [0 0 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 1]]


In [ ]:
pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())

,amazing,an,best,game,great,guys,is,of,series,should,so,the,thrones,tv,watch,watching,worth,you
0,1,1,0,1,0,0,1,1,1,0,0,0,1,1,0,0,0,0
1,0,0,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0
2,0,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0
3,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,1,1


##Tf-IDF

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

!pip install num2words
from num2words import num2words

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from nltk.corpus import stopwords
corpus = [
      'this is the first document!',
      'this document is the second document.',
      'and, he ran fast',
      'I saw him running across the road-12',
      'we are in the fifth document, right?',
      'We got played',
      'will she play with me?'
 ] 

lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocessText(doc, lem = lemmatizer):
  # lower
  lower = np.char.lower(doc)

  # stopwords
  t = []
  for i in lower:
    x = list(filter(lambda x: x not in stopwords.words('english'), i.split()))
    t.append(' '.join(x))
  
  # punctuation
  symbols = "!,\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
  for i in symbols:
    t = np.char.replace(t, i, ' ')
  
  # convert number to words; 1 to one
  p = []
  for i in t:
    x = [num2words(x) if x.isdigit() else x for x in i.split(' ')]
    p.append(' '.join(x))

  # stopwords again
  q = []
  for i in p:
    x = list(filter(lambda x: x not in stopwords.words('english'), i.split()))
    q.append(' '.join(x))

  # lemmatizing
  r = []
  for i in q:
    x = [lemmatizer.lemmatize(x, pos ='a') for x in i.split(' ')]
    r.append(' '.join(x))

  return r 

In [ ]:
new_corpus = preprocessText(corpus) 
new_corpus

['first document',
 'document second document',
 'ran fast',
 'saw running across road twelve',
 'fifth document right',
 'got played',
 'play']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer()
tfidf = tfidfvectorizer.fit_transform(new_corpus)

In [ ]:
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
tfidf_tokens

array(['across', 'document', 'fast', 'fifth', 'first', 'got', 'play',
       'played', 'ran', 'right', 'road', 'running', 'saw', 'second',
       'twelve'], dtype=object)

In [ ]:
df_tfidfvect = pd.DataFrame(data = tfidf.toarray(), index = ['Doc1','Doc2', 'Doc3', 'Doc4', 'Doc5', 'Doc6', 'Doc7'],columns = tfidf_tokens)
df_tfidfvect

,across,document,fast,fifth,first,got,play,played,ran,right,road,running,saw,second,twelve
Doc1,0.000000,0.578667,0.000000,0.000000,0.815564,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Doc2,0.000000,0.817426,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.576034,0.000000
Doc3,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.0,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Doc4,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.447214,0.447214,0.447214,0.000000,0.447214
Doc5,0.000000,0.448438,0.000000,0.632022,0.000000,0.000000,0.0,0.000000,0.000000,0.632022,0.000000,0.000000,0.000000,0.000000,0.000000
Doc6,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.0,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Doc7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##Word2Vec

In [1]:
import nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re

In [2]:
paragraph = """I have three visions for India. In 3000 years of our history, people from all over 
               the world have come and invaded us, captured our lands, conquered our minds. 
               From Alexander onwards, the Greeks, the Turks, the Moguls, the Portuguese, the British,
               the French, the Dutch, all of them came and looted us, took over what was ours. 
               Yet we have not done this to any other nation. We have not conquered anyone. 
               We have not grabbed their land, their culture, 
               their history and tried to enforce our way of life on them. 
               Why? Because we respect the freedom of others.That is why my 
               first vision is that of freedom. I believe that India got its first vision of 
               this in 1857, when we started the War of Independence. It is this freedom that
               we must protect and nurture and build on. If we are not free, no one will respect us.
               My second vision for India’s development. For fifty years we have been a developing nation.
               It is time we see ourselves as a developed nation. We are among the top 5 nations of the world
               in terms of GDP. We have a 10 percent growth rate in most areas. Our poverty levels are falling.
               Our achievements are being globally recognised today. Yet we lack the self-confidence to
               see ourselves as a developed nation, self-reliant and self-assured. Isn’t this incorrect?
               I have a third vision. India must stand up to the world. Because I believe that unless India 
               stands up to the world, no one will respect us. Only strength respects strength. We must be 
               strong not only as a military power but also as an economic power. Both must go hand-in-hand. 
               My good fortune was to have worked with three great minds. Dr. Vikram Sarabhai of the Dept. of 
               space, Professor Satish Dhawan, who succeeded him and Dr. Brahm Prakash, father of nuclear material.
               I was lucky to have worked with all three of them closely and consider this the great opportunity of my life. 
               I see four milestones in my career"""

In [16]:
# preprocessing the data

# remove all digits
data = re.sub('/d', '', paragraph)

# remove the extra spaces with one space
data = re.sub('/s+', ' ', data)

# lower
data = data.lower()

# sentence tokenize
sentences = nltk.sent_tokenize(data)

# word tokenize
tokens = [nltk.word_tokenize(sent) for sent in sentences]

# stopwords
for i,sent in enumerate(tokens):
  tokens[i] = [word for word in sent if word not in stopwords.words('english')]

In [21]:
tokens[0]

['three', 'visions', 'india', '.']

In [22]:
# training the model
# min count is which words it should, here all the words occuring atleast one time in the text.
model = Word2Vec(tokens, min_count=1)

In [27]:
# vocab 
words = model.wv.vocab
list(words)[:10]

['three',
 'visions',
 'india',
 '.',
 '3000',
 'years',
 'history',
 ',',
 'people',
 'world']

In [30]:
word_vector = model.wv['alexander']
word_vector, len(word_vector)

(array([ 2.8059552e-03,  4.9866070e-03, -1.5203004e-03, -3.5911023e-03,
        -5.8547826e-04,  5.1038555e-04,  1.5275431e-03,  4.5051728e-03,
         1.8846603e-03,  3.4140195e-03, -4.7158469e-03, -3.4616184e-03,
        -2.1153018e-03, -1.0830776e-04,  3.8605924e-03, -1.0341927e-03,
         1.5630678e-03,  3.1031484e-03, -2.5253725e-04,  2.4469832e-03,
         1.1939097e-03, -1.9765969e-03, -2.8547109e-03,  4.1097570e-03,
        -5.3475215e-04, -3.0703326e-05, -1.7723375e-03, -4.0214537e-03,
         2.4042760e-03,  1.7760998e-03,  3.6515442e-03,  3.4579760e-04,
        -9.5160876e-04,  2.3219429e-03, -4.3858197e-03,  2.9593578e-03,
        -2.9622531e-03, -3.8996660e-03, -2.3139862e-03,  4.4837487e-03,
         2.1442447e-03,  6.8988220e-04, -3.0440945e-04,  4.3706805e-03,
        -1.4550112e-03, -4.3029212e-03,  3.5659089e-03, -7.7748508e-04,
         3.7912012e-03, -3.7173270e-03, -4.6419953e-03, -3.1728880e-03,
         3.3745260e-03, -4.7932067e-03,  7.5120950e-04,  7.68832

In [32]:
# model will not give accurate result because small dataset.
similar = model.wv.most_similar('rate')
similar

[('vision', 0.25487497448921204),
 ('dutch', 0.2374258041381836),
 ('incorrect', 0.1852630376815796),
 ('respect', 0.1592533439397812),
 ('captured', 0.15523508191108704),
 ('alexander', 0.15375159680843353),
 ('milestones', 0.13519655168056488),
 ('got', 0.12959729135036469),
 ('career', 0.12948518991470337),
 ('time', 0.12394262850284576)]